In [23]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='52bc6eb2-5a84-4086-b556-6f863c699e29', project_access_token='p-26004cbd05925d2561c6f18c9f486d1016ea0e28')
pc = project.project_context

print("Project Name: {0}".format(project.get_name()))
print("Project Description: {0}".format(project.get_description()))
print("Project Bucket Name: {0}".format(project.get_project_bucket_name()))
print("Project Assets: {0}".format(project.get_assets(asset_type='connection')))

Project Name: FINAL CAPSTONE PROJECT
Project Description: Final project of the IBM Certification course on coursera
Project Bucket Name: finalcapstoneproject-donotdelete-pr-pqfq6z4hvvtutg
Project Assets: []


<h1 align=center><font size = 5> WEB SCRAPING TO COLLECT NEIGHBORHOODS INFORMATION OF BOGOTA</font></h1>

## AUTHOR: Andrés Reinoso
##### IBM Data Science Applied Capstone Project - April 2020

In [1]:
#As long as some of the datasets are available online, I will make use of the webscrapping tool BeatifulSoup in order to collect data into a dataframe. 
#This process will be done only to Bogotá dataset, but the same process may be used to obtain the data for the remaining cities.
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests

In [2]:
# Defining the website path and parser.
r = requests.get('https://es.wikipedia.org/wiki/Anexo:Barrios_de_Bogot%C3%A1')
page=bs(r.text,"html.parser")
rtable=page.table # finds the table on the html file
results=rtable.find_all('tr')

In [3]:
results[0] # Checks the format of the information

<tr>
<th>Número UPZ
</th>
<th>Unidad de Planeamiento Zonal
</th>
<th>Localidad
</th>
<th>Barrio
</th></tr>

In [4]:
## Finds the header 
header_pre=results[0].text.split()
header_pre

['Número',
 'UPZ',
 'Unidad',
 'de',
 'Planeamiento',
 'Zonal',
 'Localidad',
 'Barrio']

In [5]:
## As long as the header_pre does not represent the real header, we need to change it sligthly
header = ['Número UPZ', 'Unidad de Planeamiento Zonal','Localidad','Barrios']
header

['Número UPZ', 'Unidad de Planeamiento Zonal', 'Localidad', 'Barrios']

In [6]:
# The following code will extract all important data from the table to our dataframe df
results=rtable.find_all('tr')
nrows=len(results)
records=[]
n=1
while n < nrows:
    NumeroUPZ=results[n].text.split('\n')[1]
    Unidad_de_Planeamiento_Zonal=results[n].text.split('\n')[3]
    Localidad=results[n].text.split('\n')[5]
    Barrios=results[n].text.split('\n')[7]
    records.append((NumeroUPZ,Unidad_de_Planeamiento_Zonal,Localidad,Barrios))
    n=n+1
    
df=pd.DataFrame(records,columns=['NumeroUPZ','Unidad_de_Planeamiento_Zonal','Localidad','Barrios'])
df.head() # print initial 5 values to confirm if the result is consistent
          #however as you may notice on the column Barrios for the "0" index we can find one problem, the last Neighborhood is mixed with the word y = and


,NumeroUPZ,Unidad_de_Planeamiento_Zonal,Localidad,Barrios
0,1,Paseo de los Libertadores,01 Usaquén,"Canaima, La Floresta de La Sabana yTorca."
1,9,Verbenal,01 Usaquén,"Altos de Serrezuela, Balcones de Vista Hermosa..."
2,10,La Uribe,01 Usaquén,"Bosque de San Antonio, Conjunto Camino del Pal..."
3,11,San Cristóbal Norte,01 Usaquén,"Ainsuca, Altablanca, Barrancas, California, Ce..."
4,12,Toberín,01 Usaquén,"El Toberín, Babilonia, Darandelos, Estrella de..."


In [7]:
# Selecting all the Neighborhoods for each District and Borough.
lista_barrios = df[['Unidad_de_Planeamiento_Zonal','Localidad','Barrios']]

In [8]:
#This part of the code resolves the problem indentified to lines above by replacing the " y" with comma to have properly organized the last Neighborhood
#Also removes the last dot on the final item to have only the name by using the strip(".") method
a=" y"
for index,rows in lista_barrios.iterrows():
    print(rows['Barrios'])
    if a in rows['Barrios']:
        rows['Barrios']=rows['Barrios'].strip(".").replace(a,", ")
    else:
        rows['Barrios']=rows['Barrios'].strip(".")
    print(rows['Barrios'])

Canaima, La Floresta de La Sabana yTorca.
Canaima, La Floresta de La Sabana, Torca
Altos de Serrezuela, Balcones de Vista Hermosa, Balmoral Norte, Buenavista, Chaparral, El Codito, El Refugio de San Antonio, El Verbenal, Horizontes, La Estrellita, La Frontera, La Llanurita, Los Consuelos, Marantá, Maturín, Medellín, Mirador del Norte, Nuevo Horizonte, San Antonio Norte, Santandersito, Tibabita, Viña del Mar.
Altos de Serrezuela, Balcones de Vista Hermosa, Balmoral Norte, Buenavista, Chaparral, El Codito, El Refugio de San Antonio, El Verbenal, Horizontes, La Estrellita, La Frontera, La Llanurita, Los Consuelos, Marantá, Maturín, Medellín, Mirador del Norte, Nuevo Horizonte, San Antonio Norte, Santandersito, Tibabita, Viña del Mar
Bosque de San Antonio, Conjunto Camino del Palmar, El Pite, El Redil, La Cita, La Granja Norte, La Uribe, Los Naranjos, San Juan Bosco, Urbanización Los Laureles.
Bosque de San Antonio, Conjunto Camino del Palmar, El Pite, El Redil, La Cita, La Granja Norte, L

In [9]:
# here we use a dictionary to organize all the Barrios=Neighborhoods by each Unidad_de_Planeamiento_Zonal = Borough
dic={}
i=0
for element in range(0,112):
    dic[lista_barrios['Unidad_de_Planeamiento_Zonal'][i]]=lista_barrios['Barrios'][i]
    i=i+1

In [10]:
# Checking if the result is consistent
print(dic["La Uribe"])

Bosque de San Antonio, Conjunto Camino del Palmar, El Pite, El Redil, La Cita, La Granja Norte, La Uribe, Los Naranjos, San Juan Bosco, Urbanización Los Laureles


In [11]:
#Know we will split each value in the dictionary into a list, to have it separate and be able to execute our queries to Foursquare in the coming steps
for keys,values in dic.items():
    dic[keys]=values.split(",")

In [12]:
# Checking that information is consistent
print(dic["La Uribe"][-1])

 Urbanización Los Laureles


In [13]:
# Below two lists will be created in order to store the results for the keys and values, to be able to merge both of them into a df.
lista_a=[]
lista_b=[]
for keys,values in dic.items():
    j=0
    for i in values:
        lista_a.append(keys)
        lista_b.append(values[j])
        j=j+1 

In [14]:
# Now will change names to 'Borough' and 'Neighborhood'of the data, and will set the information into one single df.
column_names = ['Borough', 'Neighborhood']
neighborhoods = pd.DataFrame({'Borough' : lista_a,
                                'Neighborhood' : lista_b},columns=column_names)

In [15]:
#Now we are going to use exclusively the information of the neighborhood names that will be needed to perform the coordinate search and
#the explore query on foursquare
df_bogota=neighborhoods[['Neighborhood']]
df_bogota['City']= "Bogota"
df_bogota.tail()

,Neighborhood,City
1961,Las Brisas,Bogota
1962,Potosí,Bogota
1963,Las Vegas de Potosí,Bogota
1964,Villas de Bolívar,Bogota
1965,Verona,Bogota


In [17]:
# Saving the df into a csv file to be used on the coming steps!
#df_bogota.to_csv('NEIGBORHOODLISTBOG.csv',index=False) This line will work on other environments but not in Watson Studio.

In [24]:
# Save dataframe as csv file to storage
# So here we have to use project-lib for Python: If you need more information you can check https://medium.com/ibm-data-science-experience/control-your-dsx-projects-using-python-c69e13880312
project.save_data(data=df_bogota.to_csv(index=False),file_name='NEIGBORHOODLISTBOG.csv',overwrite=True)

{'file_name': 'NEIGBORHOODLISTBOG.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'finalcapstoneproject-donotdelete-pr-pqfq6z4hvvtutg',
 'asset_id': '28c399e7-c40f-488f-a4f2-85b32bee8507'}

Here ends the webscraping process, please notice we already have a dataframe with the full information of Neighborhoods of Bogota that can be use in further steps.